# 수정한 점

1. hidden layer 3으로 조정
2. Hidden unit : 2048 -> 512 -> 256
3. lr=0.0005
4. Color Jitter , Random Crop 추가하여 데이터 증강 (우성님 피드백)

## 1. STL10 데이터셋 클래스 정의
STL10 데이터셋을 학습 및 테스트 데이터로 로드하고, 파이토치 데이터로더를 통해 배치 단위로 데이터를 준비하는 작업을 수행   
해당 데이터셋을 모댈 학습에 사용할 수 있도록 전처리와 로더 생성까지 포함

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split


# 데이터 변환 정의
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 50% 확률로 가로 뒤집기
    transforms.RandomRotation(10),     # ±10도 회전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 밝기, 대비, 채도, 색조 +- 20%로 조정
    transforms.RandomCrop(80, padding=8),  # 이미지를 80x80으로 무작위 자르기 (padding 추가)
    transforms.Resize((96, 96)),  # 원래 크기인 96x96으로 복원
    transforms.ToTensor(),       # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 정규화
])

# STL10 학습 및 테스트 데이터셋 다운로드
train_dataset = datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = datasets.STL10(root='./data', split='test', transform=transform, download=True)

# Training 데이터를 Validation Set으로 나누기
train_indices, val_indices = train_test_split(range(len(train_dataset)), test_size=0.2, random_state=42)
train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(train_dataset, val_indices)

# DataLoader 생성 (64개의 배치로 나누고, 학습 데이터를 셔플)
batch_size = 64
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 2.64G/2.64G [11:35<00:00, 3.80MB/s]


Extracting ./data/stl10_binary.tar.gz to ./data
Files already downloaded and verified


In [ ]:
from torch import nn

# MLP 모델 정의
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation_function):
        super(MLP, self).__init__()

        layers = []
        previous_size = input_size

        # Hidden layers 생성
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(previous_size, hidden_size))  # Linear layer
            layers.append(nn.BatchNorm1d(hidden_size))  # 배치 정규화
            layers.append(activation_function())  # Activation function

            layers.append(nn.Dropout(0.3))  # 드롭아웃

            previous_size = hidden_size

        # Output layer 생성
        layers.append(nn.Linear(previous_size, output_size))  # 최종 출력층
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


# 모델 파라미터
input_size = 96 * 96 * 3  # 입력 크기
hidden_sizes = [2048, 512, 256]  # 점진적으로 크기 줄이도록 수정 / hidden layer의 개수는  hiddensize.length와 같음
output_size = 10  # 클래스 개수
activation_function = nn.ReLU  # 활성화 함수

# 모델 생성
model = MLP(input_size, hidden_sizes, output_size, activation_function)
print(model)


MLP(
  (model): Sequential(
    (0): Linear(in_features=27648, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=2048, out_features=512, bias=True)
    (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [ ]:
from torch import optim
from torch.optim.lr_scheduler import StepLR

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# Optimizer 설계
lr = 0.001

# 에포크 설정
epochs = 100


# 옵티마이저와 스케줄러
optimizer = optim.Adam(model.parameters(), lr)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)


# epoch 별 train loss / validation loss / accuracy loss 저장할 list --> 매번 출력하여 추이 파악
list_epoch = []
list_train_loss = []
list_val_loss = []
list_acc = []
list_acc_epoch = []




# Train
# 학습 함수 정의
def train_model(model, dataloader, val_loader, criterion, optimizer, epochs, patience=5):
    # train_losses = []  # Store losses for each epoch
    # val_losses = []  # Store validation losses for each epoch
    # accuracies = []  # Store accuracies for each epoch

    best_val_loss = float('inf')  # 초기 최상의 val_loss를 무한대로 설정
    patience_counter = 0  # Early Stopping 카운터 초기화

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in dataloader:

            images = images.view(images.size(0), -1)  # 이미지를 1D 벡터로 변환
            optimizer.zero_grad()
            outputs = model(images)  # 모델에 입력

            loss = criterion(outputs, labels)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 파라미터 업데이트
            train_loss = train_loss + loss.item()

        train_loss = train_loss / len(dataloader)
        list_train_loss.append(train_loss)
        list_epoch.append(epoch)

        # validation loss
        val_loss = evaluate_loss(model, val_loader, criterion)
        list_val_loss.append(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early Stopping 조건 확인 - val_loss가 증가하면 더이상 학습을 하지 않는 로직
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0  # 개선이 있으면 카운터 리셋
        else:
            patience_counter += 1  # 개선되지 않으면 카운터 증가
            print(f"Early Stopping Counter: {patience_counter}/{patience}")

            if patience_counter >= patience:  # patience에 도달하면 학습 종료
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        # 스케줄러 업데이트
        scheduler.step()


# loss function 평가
def evaluate_loss(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)



# Evaluation
# 평가 함수 정의
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # 평가 시에는 그래디언트 계산 비활성화
        for images, labels in dataloader:
            images = images.view(images.size(0), -1)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  # 예측 값 추출
            correct = correct + (predicted == labels).sum().item()
            total = total + labels.size(0)
    accuracy = correct / total
    list_acc.append(accuracy)
    list_acc_epoch.append(len(list_epoch))
    print(f"Accuracy: {accuracy * 100:.2f}%")
    return accuracy


In [ ]:
# 모델 학습
train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

Epoch [1/100], Train Loss: 1.7031, Val Loss: 1.6879
Epoch [2/100], Train Loss: 1.6960, Val Loss: 1.7091
Early Stopping Counter: 1/5
Epoch [3/100], Train Loss: 1.6621, Val Loss: 1.6884
Early Stopping Counter: 2/5
Epoch [4/100], Train Loss: 1.6739, Val Loss: 1.6965
Early Stopping Counter: 3/5
Epoch [5/100], Train Loss: 1.6645, Val Loss: 1.6824
Epoch [6/100], Train Loss: 1.6394, Val Loss: 1.6882
Early Stopping Counter: 1/5
Epoch [7/100], Train Loss: 1.6470, Val Loss: 1.7134
Early Stopping Counter: 2/5
Epoch [8/100], Train Loss: 1.6452, Val Loss: 1.6973
Early Stopping Counter: 3/5
Epoch [9/100], Train Loss: 1.6541, Val Loss: 1.6759
Epoch [10/100], Train Loss: 1.6322, Val Loss: 1.6738
Epoch [11/100], Train Loss: 1.6447, Val Loss: 1.6737
Epoch [12/100], Train Loss: 1.6298, Val Loss: 1.6622
Epoch [13/100], Train Loss: 1.6200, Val Loss: 1.6703
Early Stopping Counter: 1/5
Epoch [14/100], Train Loss: 1.6279, Val Loss: 1.6718
Early Stopping Counter: 2/5
Epoch [15/100], Train Loss: 1.6049, Val Los

In [12]:
# 모델 평가
accuracy = evaluate_model(model, test_loader)

Accuracy: 41.76%


# 결과물 기록
hidden_sizes = [2048, 512, 256] / hidden layer = 3 / lr = 0.001 / 100 epoch / Act functino : \ReLU / Dropout : 0.3 / scheduler = StepLR(optimizer, step_size=20, gamma=0.5)- Accuracy: 48.89%